<a href="https://colab.research.google.com/github/dabin3178/KoBERT_Multi-label/blob/main/%EB%8B%A4%EB%B9%88_kobert_multi_label_%ED%99%95%EB%A5%A0(sigmoid)%ED%8F%AC%ED%95%A8_230516.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python --version

Python 3.10.11


In [2]:
# 설치 후 런타임 다시 시작 필요할 수도 (런타임 다시 시작 필요하면 실행창에 뜸)
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 15.8 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.8.0-py3-none-any.whl size=292701 sha256=f175bf403084e3ba7b16db99133628dc8f1f28d02f5b5f0479ac78b74c24339a
  Stored in directory: /root/.cache/pip/wheels/2d/cc/dc/7ec84dced25f738b8be400101abb67e4b50c905090a51017e4
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, ht

In [3]:
# Hugging Face API로 KoBERT 다운로드
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-q4p1mrjd/kobert-tokenizer_770135a75e934ec3bb5bfab732d245c2
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-q4p1mrjd/kobert-tokenizer_770135a75e934ec3bb5bfab732d245c2
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=5775330f86873d418dde871606900dac32b24c2a01100736ca5962c20be25ff0
  Stored in directory: /tmp/pip-ephem-wheel-cache-pkao9ygj/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


In [5]:
!pip3 install kobert-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
# ★ Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
# ●
#from transformers import BertTokenizer

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [7]:
# GPU 사용 확인
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [8]:
# GPU 사용 시
device = torch.device("cuda:0")

In [9]:
# ★
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')
# ●
#bert_tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [10]:
# 파라미터 세팅
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

#안 돌림1

In [ ]:
import pandas as pd

# 웰니스 데이터 시트별로 라벨링해 저장
wellness = pd.DataFrame(columns=['상담 데이터', 'label'])

for i in range(9):
  globals()['wellness{}'.format(i+1)] = pd.read_excel('/content/drive/MyDrive/LAMDA/웰니스 DSM-5 분류.xlsx', sheet_name = i)
  globals()['wellness{}'.format(i+1)]['label'] = i
  wellness = wellness.append(globals()['wellness{}'.format(i+1)][['상담 데이터', 'label']])

<ipython-input-11-843c516d101a>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  wellness = wellness.append(globals()['wellness{}'.format(i+1)][['상담 데이터', 'label']])
<ipython-input-11-843c516d101a>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  wellness = wellness.append(globals()['wellness{}'.format(i+1)][['상담 데이터', 'label']])
<ipython-input-11-843c516d101a>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  wellness = wellness.append(globals()['wellness{}'.format(i+1)][['상담 데이터', 'label']])
<ipython-input-11-843c516d101a>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  wellness = wellness.append(globals()['wellness{}'.format(i+1)][['

In [11]:
# 웰니스 데이터로 KoBERT 훈련을 위한 데이터 형태 만들기 

data_list = []

for q, label in zip(wellness['상담 데이터'], wellness['label'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

NameError: ignored

In [ ]:
data

NameError: ignored

In [ ]:
data_list

[['아이 가졌는데 기분 하나도 안 좋고 울적해', '0'],
 ['아이 가졌는데 기분 하나도 안 좋고 울적해', '0'],
 ['임신했는데 남편이 하나도 안 챙겨줘서 우울하다', '0'],
 ['하고 싶은 거 못 해서 이렇게 우울할 수가 없어', '0'],
 ['술 못 마시니까 진짜 우울하다', '0'],
 ['유도 실패해서 개 우울해', '0'],
 ['기분이 계속 가라앉아서 땅 속에 꺼진 느낌이야', '0'],
 ['밤에 특히 더 우울하다', '0'],
 ['우울하니까 눈물 나고..', '0'],
 ['착잡하니까 울고 싶고 그래', '0'],
 ['우울증 때문에 죽고 싶어', '0'],
 ['우울해 죽고 싶어요', '0'],
 ['우울해서 힘드네', '0'],
 ['우울한 건 어쩔 수가 없어', '0'],
 ['우울한 건 어쩔 수 없어', '0'],
 ['계속 우울하다', '0'],
 ['이유없이 우울하다', '0'],
 ['오빠 때문에 침울해', '0'],
 ['엄마 때문에 침울해요', '0'],
 ['요새 되는 일이 없어서 침울한 것 같아', '0'],
 ['단음식도 전부 끊었는데 임당 재검 떠서 침울해', '0'],
 ['노산이라 침울해져', '0'],
 ['대화가 안 통하니까 침울하지', '0'],
 ['임산부라고 쳐다보는 시선 때문에 침울해져요', '0'],
 ['대화가 안 통하니까 세상 침울합니다', '0'],
 ['오늘 상담 받고 왔는데 진짜 침울해지는 것 같아', '0'],
 ['유도 실패해서 침울해', '0'],
 ['유도 실패해서 침울해', '0'],
 ['재검 해야 한다고 해서 침울하다', '0'],
 ['엄마 자격이 없는 것 같아서 침울해', '0'],
 ['조기수축 때문에 퇴원을 못하니까 침울해져', '0'],
 ['암흑 속에 있는 것처럼 침울해', '0'],
 ['침울하니까 머리도 아픈 것 같아', '0'],
 ['하루하루 지나니까 더 침울해지는 것 같아', '0'],
 ['침울해서 돈을 쓰게 돼', '0'],
 ['침울하니까 안

In [ ]:
# 훈련, 테스트 데이터셋 분리 
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

# 여기서부터 다시 돌림

In [12]:
data_pd = pd.read_csv('/content/drive/MyDrive/LAMDA/mutil_labeled_wellness_data.csv')
data_pd

,Unnamed: 0,상담 데이터,1,2,3,4,5,6,7,8,9
0,0,기분이 한없이 우울해지고 잠도 사라지고 의욕도 없어,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,너무너무 슬프고 너무너무 힘들어,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,임신하고 나서 우울하고 의욕이 없어,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,3,하루 종일 어디나 가지도 못하니까 눈물이 나는 거야,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,하루하루가 지옥 같아,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
9779,9779,힘이 없어서 사소한 일도 못 하겠어,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9780,9780,힘이 없어서 아무것도 안 하게 돼,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9781,9781,힘이 없어서 애한테 화내게 돼,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9782,9782,힘이 없어서 우울해,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [13]:
data_pd = data_pd.drop(data_pd.columns[[0]],axis=1)
data_pd

,상담 데이터,1,2,3,4,5,6,7,8,9
0,기분이 한없이 우울해지고 잠도 사라지고 의욕도 없어,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,너무너무 슬프고 너무너무 힘들어,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,임신하고 나서 우울하고 의욕이 없어,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,하루 종일 어디나 가지도 못하니까 눈물이 나는 거야,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,하루하루가 지옥 같아,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
9779,힘이 없어서 사소한 일도 못 하겠어,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9780,힘이 없어서 아무것도 안 하게 돼,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9781,힘이 없어서 애한테 화내게 돼,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9782,힘이 없어서 우울해,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [14]:
# 굳이 안 필요한 과정일 수도?
data_pd = data_pd.astype({'1':int, '2':int, '3':int, '4':int, '5':int, '6':int, '7':int, '8':int, '9':int})
data_pd

,상담 데이터,1,2,3,4,5,6,7,8,9
0,기분이 한없이 우울해지고 잠도 사라지고 의욕도 없어,0,1,0,0,0,0,0,0,0
1,너무너무 슬프고 너무너무 힘들어,1,1,0,0,0,0,0,0,0
2,임신하고 나서 우울하고 의욕이 없어,0,1,0,0,0,0,0,0,1
3,하루 종일 어디나 가지도 못하니까 눈물이 나는 거야,1,0,0,0,0,0,0,0,0
4,하루하루가 지옥 같아,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
9779,힘이 없어서 사소한 일도 못 하겠어,0,0,0,0,0,1,0,0,0
9780,힘이 없어서 아무것도 안 하게 돼,0,0,0,0,0,1,0,0,0
9781,힘이 없어서 애한테 화내게 돼,0,0,0,0,0,1,0,0,0
9782,힘이 없어서 우울해,0,0,0,0,0,1,0,0,0


In [15]:
data_pd_list = data_pd.to_numpy()
data_pd_list

array([[' 기분이 한없이 우울해지고 잠도 사라지고 의욕도 없어', 0, 1, ..., 0, 0, 0],
       [' 너무너무 슬프고 너무너무 힘들어', 1, 1, ..., 0, 0, 0],
       [' 임신하고 나서 우울하고 의욕이 없어', 0, 1, ..., 0, 0, 1],
       ...,
       ['힘이 없어서 애한테 화내게 돼', 0, 0, ..., 0, 0, 0],
       ['힘이 없어서 우울해', 0, 0, ..., 0, 0, 0],
       ['힘이 없어서 죽겠어', 0, 0, ..., 0, 0, 0]], dtype=object)

In [16]:
# 훈련, 테스트 데이터셋 분리 
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_pd_list, test_size=0.25, random_state=0)

In [17]:
dataset_train

array([['짜증을 주체할 수가 없으니까 다른 사람을 막 때리려고 해', 1, 1, ..., 0, 0, 0],
       ['근데 한동안 나 스스로가 너무 자괴감이 들어서 관뒀어.', 0, 0, ..., 1, 0, 0],
       ['제왕절개 얘기하셔서 너무 괴롭고 절망적이야', 1, 0, ..., 0, 0, 0],
       ...,
       ['성적 너무 많이 떨어져서 걱정돼…', 0, 0, ..., 0, 1, 0],
       ['마음을 추스르기가 힘들어', 1, 1, ..., 0, 0, 0],
       ['노력하기 전에 포기부터 해버려서 이젠 뭘 어떻게 노력해야 하는지 잘 모르겠어.', 1, 1, ..., 0, 0,
        0]], dtype=object)

In [135]:
# 새연's ver.
class BERTSentenceTransform:
    r"""BERT style data transformation.

    Parameters
    ----------
    tokenizer : BERTTokenizer.
        Tokenizer for the sentences.
    max_seq_length : int.
        Maximum sequence length of the sentences.
    pad : bool, default True
        Whether to pad the sentences to maximum length.
    pair : bool, default True
        Whether to transform sentences or sentence pairs.
    """

    def __init__(self, tokenizer, max_seq_length,vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab 

    def __call__(self, line):
        """Perform transformation for sequence pairs or single sequences.

        The transformation is processed in the following steps:
        - tokenize the input sequences
        - insert [CLS], [SEP] as necessary
        - generate type ids to indicate whether a token belongs to the first
        sequence or the second sequence.
        - generate valid length

        For sequence pairs, the input is a tuple of 2 strings:
        text_a, text_b.

        Inputs:
            text_a: 'is this jacksonville ?'
            text_b: 'no it is not'
        Tokenization:
            text_a: 'is this jack ##son ##ville ?'
            text_b: 'no it is not .'
        Processed:
            tokens: '[CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]'
            type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
            valid_length: 14

        For single sequences, the input is a tuple of single string:
        text_a.

        Inputs:
            text_a: 'the dog is hairy .'
        Tokenization:
            text_a: 'the dog is hairy .'
        Processed:
            text_a: '[CLS] the dog is hairy . [SEP]'
            type_ids: 0     0   0   0  0     0 0
            valid_length: 7

        Parameters
        ----------
        line: tuple of str
            Input strings. For sequence pairs, the input is a tuple of 2 strings:
            (text_a, text_b). For single sequences, the input is a tuple of single
            string: (text_a,).

        Returns
        -------
        np.array: input token ids in 'int32', shape (batch_size, seq_length)
        np.array: valid length in 'int32', shape (batch_size,)
        np.array: input token type ids in 'int32', shape (batch_size, seq_length)

        """

        # convert to unicode
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a)
        #tokens_a = self._tokenizer(text_a)
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        # The embedding vectors for `type=0` and `type=1` were learned during
        # pre-training and are added to the wordpiece embedding vector
        # (and position vector). This is not *strictly* necessary since
        # the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.

        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        #vocab = self._tokenizer.vocab
        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        # The valid length of sentences. Only real  tokens are attended to.
        valid_length = len(input_ids)

        if self._pad:
            # Zero-pad up to the sequence length.
            padding_length = self._max_seq_length - valid_length
            # use padding tokens for the rest
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')
#[출처] No module named 'kobert' 에러 해결|작성자 yeon



In [136]:
# 새연's ver.
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        #transform = nlp.data.BERTSentenceTransform(
        #    tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

#저는 csv 파일을 사용하여 이부분은 dataset_train.values 로 해주었습니다. 
data_train = BERTDataset(dataset_train, 0, 1, tokenizer, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tokenizer, vocab, max_len, True, False)
#[출처] No module named 'kobert' 에러 해결|작성자 yeon

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [137]:
# 데이터로더 생성
batch_size = 64
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
# 혜진's ver. -> 미안함니다,, 근데 새연's ver. 쓰는 게 다른 문서 참고하기가 더 편할 것 같아요,,,
'''
class BERTDataset(Dataset):
    def __init__(self, dataset, tokenizer, max_len, pad=True, pair=False):
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.pad = pad
        self.pair = pair
        self.dataset = dataset

    def __getitem__(self, idx):
        if self.pair:
            text = self.dataset[idx][0][0]
            text_pair = self.dataset[idx][0][1]
        else:
            text = self.dataset[idx][0]

        inputs = self.tokenizer.encode_plus(
            text,
            text_pair if self.pair else None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=self.pad,
            truncation=False
        )

        item = {
            'input_ids': torch.tensor(inputs['input_ids']),
            'attention_mask': torch.tensor(inputs['attention_mask']),
            'token_type_ids': torch.tensor(inputs['token_type_ids'])
        }

        item['labels'] = int(self.dataset[idx][1])

        # 패딩된 시퀀스/길이와 타입에 대한 내용/어텐션 마스크 시퀀스 세 가지 배열을 얻을 수 있도록 함
        padded_seq = torch.tensor(inputs['input_ids']).numpy()
        len_and_type = list(torch.tensor(inputs['token_type_ids']).shape) #dtype은 출력 안됨
        attention_mask_seq = torch.tensor(inputs['attention_mask']).numpy()

        return np.array(padded_seq), np.array(len_and_type), np.array(attention_mask_seq), item['labels']


    def __len__(self):
        return len(self.dataset)


# 데이터셋 생성
train_dataset = BERTDataset(dataset_train, tokenizer, max_len, True, True)
test_dataset = BERTDataset(dataset_test, tokenizer, max_len, True, True)

# 데이터로더 생성
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
'''

In [ ]:
#train_dataset

In [138]:
data_train[0]

(array([   2, 4396, 7316, 7088, 4213, 7436, 7836, 2872, 5330, 3270, 7076,
        5591, 1567, 2584, 7088, 1927, 1844, 6122, 6061, 4998,    3,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(21, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 1)

In [139]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=9, # 라벨 수만큼 적어주기 (여기가!!!!!! 왜 !!!!!! 2였던 거야!!!!!!! 한참 헤맸잖아!!!!!!!)
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [140]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [141]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [142]:
# when 새연's
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# when 혜진's
'''
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_loader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)
'''

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [143]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [144]:
# 테스트용 데이터셋
test_df = pd.DataFrame({'articles':['우울해서 죽고싶어', '잠이 안 와', '요즘 자꾸 살쪄', '불안하고 심장이 두근거려', '약 먹은 것처럼 멍해']})
test_df

,articles
0,우울해서 죽고싶어
1,잠이 안 와
2,요즘 자꾸 살쪄
3,불안하고 심장이 두근거려
4,약 먹은 것처럼 멍해


In [145]:
# 테스트용 데이터셋
test_df = pd.DataFrame({'상담 데이터':['우울해서 죽고싶어', '잠이 안 와', '요즘 자꾸 살쪄', '불안하고 심장이 두근거려', '약 먹은 것처럼 멍해']}, columns=data_pd.columns)
test_df

,상담 데이터,1,2,3,4,5,6,7,8,9
0,우울해서 죽고싶어,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,잠이 안 와,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,요즘 자꾸 살쪄,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,불안하고 심장이 두근거려,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,약 먹은 것처럼 멍해,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [146]:
# 나중에 돌려보기 

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-146-93aef3cb4d12>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/115 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


epoch 1 batch id 1 loss 2.2601511478424072 train acc 0.0625
epoch 1 train acc 0.6852484472049689


<ipython-input-146-93aef3cb4d12>:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


...

epoch 4 test acc 0.9763621794871795


  0%|          | 0/115 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.11158542335033417 train acc 0.984375
epoch 5 train acc 0.9972826086956522


  0%|          | 0/39 [00:00<?, ?it/s]

epoch 5 test acc 0.9803685897435898


In [128]:
# KoBERT 모델로 멀티 라벨 결과 예측하기

import numpy as np
import math

# 난 소프트맥스 안 쓰니까 주석처리 하겠어요
'''
def softmax(X):
  exp_a = np.exp(X)
  sum_exp_a = np.sum(exp_a)
  y = exp_a / sum_exp_a   
  return y
'''  

def predict(df):

    for i in range(0, len(df)): # 게시글 총 문장 개수만큼 반복
      if i % 100 == 0:
        print(i)
        
      data = [df.iloc[i, 0], '0']
      #data = [df.iloc[i, 0]]
      print(data)
      dataset_another = [data]
      another_test = BERTDataset(dataset_another, 0,1, tokenizer,vocab, max_len, True, False)
      test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, shuffle=False)
      

      model.eval()

      for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
          token_ids = token_ids.long().to(device)
          segment_ids = segment_ids.long().to(device)

          valid_length= valid_length
          label = label.long().to(device)

          out = model(token_ids, valid_length, segment_ids)

          for j in out:
            #logits = j
            probability = torch.sigmoid(j).cpu().detach().numpy() # 시그모이드 함수 사용해 0과 1 사이값으로 변환
            #print(torch.sigmoid(j))

            # 시그모이드 변환값이 0.5이상이라면 라벨 할당
            temp = [] # 라벨 저장할 리스트
            for p in probability:
              if p > 0.5:
                temp.append(1)
              else:
                temp.append(0)
            print(temp)

            df.iloc[i, 1:] = temp
            #df.iloc[i, 1:] = temp[:len(df.columns)-1]
            #nn =(df.shape[1]-1)/ 2
            #df.iloc[i, 1:] = np.tile(temp, (1, nn+1))
            #df.iloc[i, 1:len(temp)+1] = temp
            
            #logits = logits.detach().cpu().numpy() # 라벨 예측 결과
            
            #df.iloc[i, 1:] = np.argmax(logits)
            #print(df.iloc[i, 1])

In [32]:
# 돌릴 필요 없음.
out

tensor([[ 3.2035, -2.9330],
        [-3.2957,  3.0831],
        [ 3.2900, -3.0308],
        [ 3.3095, -3.0615],
        [-3.3028,  3.0410],
        [-3.3005,  3.0328],
        [ 3.0251, -2.8040],
        [ 3.2534, -3.0219],
        [ 3.1540, -2.9508],
        [-3.2318,  2.9855],
        [-3.3055,  3.0903],
        [ 3.1926, -2.9835],
        [ 3.2023, -2.9872],
        [ 2.8903, -2.7060]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [129]:
predict(test_df)

0
['우울해서 죽고싶어', '0']
[0, 1, 0, 1, 0, 0, 0, 0, 0]
['잠이 안 와', '0']
[0, 1, 0, 1, 0, 0, 0, 0, 0]
['요즘 자꾸 살쪄', '0']
[0, 1, 0, 1, 0, 0, 0, 0, 0]
['불안하고 심장이 두근거려', '0']
[0, 1, 0, 1, 0, 0, 0, 0, 0]
['약 먹은 것처럼 멍해', '0']
[0, 1, 0, 1, 0, 0, 0, 0, 0]


# 안 돌림2

In [ ]:
test_df3 = pd.read_csv('/content/drive/MyDrive/LAMDA/mutil_labeled_wellness_data.csv')
test_df3

In [ ]:
test_df4 = test_df3.iloc[:5]
test_df4

In [ ]:
test_df4 = test_df4.drop(test_df4.columns[[0]],axis=1)
test_df4

In [ ]:
predict(test_df)

0
['우울해서 죽고싶어', '0', '0', '0', '0', '0', '0', '0', '0', '0']
[0, 1]


ValueError: ignored

In [ ]:
predict(test_df2)

0
['우울해서 죽고싶어', '0', '1', '2', '3', '4', '5', '6', '7', '8']
[0, 1]


ValueError: ignored

In [ ]:
predict(data_test)

0


AttributeError: ignored

In [ ]:
test_df5 = test_df4.to_numpy()
test_df5

array([[' 기분이 한없이 우울해지고 잠도 사라지고 의욕도 없어', 0.0, 1.0, 0.0, 0.0, 0.0, 0.0,
        0.0, 0.0, 0.0],
       [' 너무너무 슬프고 너무너무 힘들어', 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
        0.0],
       [' 임신하고 나서 우울하고 의욕이 없어', 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
        1.0],
       [' 하루 종일 어디나 가지도 못하니까 눈물이 나는 거야', 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,
        0.0, 0.0, 0.0],
       [' 하루하루가 지옥 같아', 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]],
      dtype=object)

In [ ]:
predict(test_df4)

0


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


[1, 0]


ValueError: ignored

In [ ]:
predict(test_df5)

0


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


[1, 0]


AttributeError: ignored

In [130]:
# KoBERT 모델로 예측한 에브리타임 게시물 분류 결과 저장 

import numpy as np

def softmax(X):
  exp_a = np.exp(X)
  sum_exp_a = np.sum(exp_a)
  y = exp_a / sum_exp_a   
  return y

def predict(df):

    for i in range(0, len(df)): # 게시글 총 문장 개수만큼 반복
      if i % 100 == 0:
        print(i)
      data = [df[i, 0], '0']
      #data = [df.iloc[i, 0]]
      print(data)
      dataset_another = [data]
      another_test = BERTDataset(dataset_another, 0,1, tokenizer,vocab, max_len, True, False)
      test_dataloader = DataLoader(another_test, batch_size=batch_size, shuffle=False)
      
      model.eval()

      for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
          token_ids = token_ids.long().to(device)
          segment_ids = segment_ids.long().to(device)

          valid_length= valid_length
          label = label.long().to(device)

          out = model(token_ids, valid_length, segment_ids)

          for j in out:
            logits = j
            probability = torch.sigmoid(j).cpu().detach().numpy() # 시그모이드 함수 사용해 0과 1 사이값으로 변환
            #print(torch.sigmoid(j))

            # 시그모이드 변환값이 0.5이상이라면 라벨 할당
            temp = [] # 라벨 저장할 리스트
            for p in probability:
              if p > 0.5:
                temp.append(1)
              else:
                temp.append(0)
            print(temp)

            df[i, 1:] = temp
            #df.iloc[i, 1:] = temp[:len(df.columns)-1]
            
            # logits = logits.detach().cpu().numpy() # 라벨 예측 결과
            
            # df.iloc[i, 1] = np.argmax(logits)

In [ ]:
predict(test_df5)

0
[' 기분이 한없이 우울해지고 잠도 사라지고 의욕도 없어', '0']
[1, 1]


ValueError: ignored

In [ ]:
test_df4 = test_df4.astype({'1':int, '2':int, '3':int, '4':int, '5':int, '6':int, '7':int, '8':int, '9':int})
test_df4

,상담 데이터,1,2,3,4,5,6,7,8,9
0,기분이 한없이 우울해지고 잠도 사라지고 의욕도 없어,0,1,0,0,0,0,0,0,0
1,너무너무 슬프고 너무너무 힘들어,1,1,0,0,0,0,0,0,0
2,임신하고 나서 우울하고 의욕이 없어,0,1,0,0,0,0,0,0,1
3,하루 종일 어디나 가지도 못하니까 눈물이 나는 거야,1,0,0,0,0,0,0,0,0
4,하루하루가 지옥 같아,1,0,0,0,0,0,0,0,0


In [ ]:
predict(test_df4)

0
[' 기분이 한없이 우울해지고 잠도 사라지고 의욕도 없어', '0']
[1, 1]


AttributeError: ignored

# 각 라벨별 확률 구하기 (Sigmoid)

In [132]:
# KoBERT 모델로 예측한 에브리타임 게시물 분류 결과 저장_확률편

import numpy as np
'''
def softmax(X):
  exp_a = np.exp(X)
  sum_exp_a = np.sum(exp_a)
  y = exp_a / sum_exp_a   
  return y
'''
def predict_prb(df):

    for i in range(0, len(df)): # 게시글 총 문장 개수만큼 반복
      if i % 100 == 0:
        print(i)
      data = [df.iloc[i, 0], '0']
      #data = [df.iloc[i, 0]]
      print(data)
      dataset_another = [data]
      another_test = BERTDataset(dataset_another, 0,1, tokenizer,vocab, max_len, True, False)
      test_dataloader = DataLoader(another_test, batch_size=batch_size, shuffle=False)
      
      model.eval()

      for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
          token_ids = token_ids.long().to(device)
          segment_ids = segment_ids.long().to(device)

          valid_length= valid_length
          label = label.long().to(device)

          out = model(token_ids, valid_length, segment_ids)

          for j in out:
            logits = j
            probability = torch.sigmoid(j).cpu().detach().numpy() # 시그모이드 함수 사용해 0과 1 사이값으로 변환
            #print(torch.sigmoid(j))
            print(probability)

'''
            # 시그모이드 변환값이 0.5이상이라면 라벨 할당
            temp = [] # 라벨 저장할 리스트
            for p in probability:
              if p > 0.5:
                temp.append(1)
              else:
                temp.append(0)
            print(temp)
'''

            #df.iloc[i, 1:] = temp
            #df.iloc[i, 1:] = temp[:len(df.columns)-1]
            
            # logits = logits.detach().cpu().numpy() # 라벨 예측 결과
            
            # df.iloc[i, 1] = np.argmax(logits)

'\n            # 시그모이드 변환값이 0.5이상이라면 라벨 할당\n            temp = [] # 라벨 저장할 리스트\n            for p in probability:\n              if p > 0.5:\n                temp.append(1)\n              else:\n                temp.append(0)\n            print(temp)\n'

In [134]:
# 테스트 진행
predict_prb(test_df)

0
['우울해서 죽고싶어', '0']
[0.47481054 0.64586914 0.44258428 0.5120275  0.43271443 0.40089142
 0.4048323  0.45361212 0.4853295 ]
['잠이 안 와', '0']
[0.47481054 0.64586914 0.44258428 0.5120275  0.43271443 0.40089142
 0.4048323  0.45361212 0.4853295 ]
['요즘 자꾸 살쪄', '0']
[0.47481054 0.64586914 0.44258428 0.5120275  0.43271443 0.40089142
 0.4048323  0.45361212 0.4853295 ]
['불안하고 심장이 두근거려', '0']
[0.47481054 0.64586914 0.44258428 0.5120275  0.43271443 0.40089142
 0.4048323  0.45361212 0.4853295 ]
['약 먹은 것처럼 멍해', '0']
[0.47481054 0.64586914 0.44258428 0.5120275  0.43271443 0.40089142
 0.4048323  0.45361212 0.4853295 ]


# 안 돌림3

In [ ]:
predict_prb(test_df4)

0
[' 기분이 한없이 우울해지고 잠도 사라지고 의욕도 없어', '1']
[0.49568838 0.47472095]
[' 너무너무 슬프고 너무너무 힘들어', '1']
[0.4984233 0.478273 ]
[' 임신하고 나서 우울하고 의욕이 없어', '1']
[0.5188175  0.53185874]
[' 하루 종일 어디나 가지도 못하니까 눈물이 나는 거야', '1']
[0.48233688 0.46601948]
[' 하루하루가 지옥 같아', '1']
[0.49996734 0.47068298]


In [ ]:
predict_prb(test_df)

0
['우울해서 죽고싶어', '0']
[0.4527498  0.59693456]
['잠이 안 와', '0']
[0.43722448 0.6092435 ]
['요즘 자꾸 살쪄', '0']
[0.41970548 0.56073153]
['불안하고 심장이 두근거려', '0']
[0.5033018 0.5268942]
['약 먹은 것처럼 멍해', '0']
[0.4448171  0.40113458]


In [ ]:
test_df

,상담 데이터
0,우울해서 죽고싶어
1,잠이 안 와
2,요즘 자꾸 살쪄
3,불안하고 심장이 두근거려
4,약 먹은 것처럼 멍해
